# Subregion Slippage / Acceleration analysis (for SLM/FLM coaching)

- Author: Luis Herrera (luis.herrera@databricks.com) 
- Version: 1.0


This notebook is designed for subregion-wide risk and acceleration analysis of of an org opportunities/ use cases, primarily to support SLM/FLM (Sales/Field Leadership) coaching and pipeline management. It leverages both data engineering and AI-powered insights to identify at-risk opportunities and recommend actions for acceleration.

**Key Features and Workflow:**

- Data Loading & Filtering:
The notebook loads curated use case data from core_usecase_curated and applies subregion filters (e.g., SEMEA) to focus the analysis.
- Risk & Confidence Scoring:
It uses AI models to score each use case’s likelihood of conversion, extract top risk drivers, and recommend actions (Proceed, Pivot, Walk) with explanations.
-Slippage & Acceleration Analysis:
The notebook identifies use cases likely to slip in the next 6 months (low confidence, near-term go-live) and quantifies the $DBU at risk, both in total and by field manager.
- Acceleration Opportunities:
For each use case, it generates AI-driven recommendations to accelerate progress, classifies them into categories (e.g., Funding, Stakeholder Engagement), and aggregates potential $DBU impact by manager and category.
- Visualization:
Results are displayed as interactive tables, supporting further dashboarding and reporting.

This workflow enables proactive pipeline management, helping leadership focus on high-impact interventions and track progress across teams and categories.

In [0]:
# Set the execution timeout to 4 hours (14400 seconds)
# Note: On serverless notebooks, the default timeout is 2.5 hours. 
# You can manually set a longer timeout (e.g., 4 hours) using this config, 
# but actual enforcement may depend on workspace and cluster policies.
spark.conf.set("spark.databricks.execution.timeout", "14400")

In [0]:
from pyspark.sql.functions import lit

# Define the fiscal year 2026 start and end dates
fy26_start_date = lit("2025-02-01")
fy26_end_date = lit("2026-01-31")

In [0]:
def subregion_wide_df(df, subregion1, subregion2, subregion3): 
    # Check the values of subregions are not None
    filters = []
    if subregion1 is not None:
        filters.append(col("sales_subregion_level_1") == subregion1)
    if subregion2 is not None:
        filters.append(col("sales_subregion_level_2") == subregion2)
    if subregion3 is not None:
        filters.append(col("sales_subregion_level_3") == subregion3)
    if not filters:
        # if nothing is provided, default to looking at EMEA (note: applying NULL bug filter to catch all EMEA)
        filters.append((col("sales_region") == 'EMEA') | (col('business_unit') == 'EMEA') | (col('field_leader') == 'Toby Balfre'))

    combined_filter = filters[0]
    for f in filters[1:]:
        combined_filter = combined_filter & f
    
    # Filter the DataFrame for usecases created or still open in FY26
    return df.filter(
        (
            combined_filter
        ) & (
            # and we want use cases modified in FY26
            (col("last_modified_date") >= fy26_start_date) & (col("last_modified_date") <= fy26_end_date)
        )
    )

In [0]:
# Set subregion filters for data processing.
# These variables can be used to filter data by subregion as needed.

subregion1 = None
subregion2 = None
subregion3 = None

In [0]:
from pyspark.sql.functions import col

# Define the columns to select from the dataframe required for the confidence analsis
cols = [
    "snapshot_date", 
    "field_manager",
    "account_name", 
    "usecase_name", 
    "account_executive",
    "account_solution_architect",
    "stage",
    "target_live_date",
    "onboarding_date",
    "estimated_monthly_dollar_dbus", 
    "usecase_description",
    "demand_plan_stage_next_steps", 
    "implementation_notes",
    "sales_subregion_level_1",
    "sales_subregion_level_2",
    "sales_subregion_level_3"
]

# Load the curated use case data from the main table
usecase_curated_df = spark.table("main.gtm_data.core_usecase_curated")

# Transform the dataframe to a wide format based on subregion filters
full_usecase_curated_df = subregion_wide_df(usecase_curated_df,
                                            subregion1, 
                                            subregion2, 
                                            subregion3)

# Select only the specified columns for further analysis
full_usecase_curated_df = full_usecase_curated_df.select(cols)

display(full_usecase_curated_df.limit(15))

**Confidence Scoring of Use Cases & Recommended Actions**

In [0]:
from pyspark.sql.functions import col, current_date, add_months, expr, trim, regexp_extract, split, when

# Helper function to escape single quotes and newlines in AI prompts
def escape_prompt(prompt):
    return prompt.replace("'", "''").replace("\n", " ")

# Filter and prepare the use case dataframe for confidence analysis
usecases_goinglive_next = (
    full_usecase_curated_df
    .select(*cols)  # Select relevant columns
    # Fill missing field_manager and account_solution_architect with default values
    .withColumn("field_manager", when(col("field_manager").isNull(), "FLM not assigned").otherwise(col("field_manager")))
    .withColumn("account_solution_architect", when(col("account_solution_architect").isNull(), "SA not assigned").otherwise(col("account_solution_architect")))
    # Apply business filters:
    # - Only use cases with estimated DBUs >= 5000
    # - Exclude certain sales stages
    # - Exclude use cases related to Unity Catalog, UC, or Governance
    # - Only include use cases with target live date in the next 6 months
    .filter(
        (col("estimated_monthly_dollar_dbus") >= 5000) &
        (~col("stage").isin(["U1", "Lost", "Closed", "Disqualified"])) &
        (~col("usecase_name").contains("Unity Catalog")) & 
        (~col("usecase_name").contains("UC")) & 
        (~col("usecase_name").contains("Governance")) &
        (col("target_live_date").between(current_date(), add_months(current_date(), 6)))
    )
    # Order by estimated DBUs descending
    .orderBy(col("estimated_monthly_dbus").desc())
)
display(usecases_goinglive_next.limit(15))

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

usecases_goinglive_next = usecases_goinglive_next.withColumn(
    "unique_id", monotonically_increasing_id()
)

usecases_goinglive_next.count()  # Force materialization and unique_id assignment

display(usecases_goinglive_next.limit(15))

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, ceil, lit, col, expr


# Applies a Spark SQL expression to a DataFrame in batches, adding the result as a new column.
# This is useful for AI functions with batch limits but also to improve performance


def batch_apply_expr(
    df,
    field_to_add: str,
    expr_str: str,
    batch_size: int = 100,
):
    from pyspark.sql import functions as F
    from pyspark.sql.window import Window

    if "unique_id" not in df.columns:
        raise ValueError("unique_id column is required")

    # Create a stable row index without a global sort: order within hash partitions by unique_id
    w = Window.partitionBy(F.spark_partition_id()).orderBy("unique_id")
    df_idx = df.withColumn("_row_num", F.row_number().over(w) - F.lit(1))

    # Derive a synthetic batch id; then range-repartition to keep partitions ≈ batch_size
    df_tagged = df_idx.withColumn("_batch_id", (F.col("_row_num") / F.lit(batch_size)).cast("int"))
    df_part = df_tagged.repartitionByRange(F.col("_batch_id"))

    # Column pruning: keep only columns needed for the expression + unique_id + housekeeping
    # Heuristic: extract candidate column names mentioned in expr_str; fall back to keeping all.
    import re as _re
    mentioned = set(_re.findall(r"\b([A-Za-z_][A-Za-z0-9_]*)\b", expr_str))
    keep = [c for c in df_part.columns if c in (mentioned | {"unique_id","_row_num","_batch_id"})] or df_part.columns
    slim = df_part.select(*keep)

    # Apply the expression in a single pass
    out = slim.withColumn(field_to_add, F.expr(expr_str))

    # Join back to any dropped columns by unique_id if we pruned aggressively
    if set(out.columns) != set(df_part.columns + [field_to_add]):
        out = out.join(df_part.drop(*[c for c in df_part.columns if c in out.columns and c != "unique_id"]), on="unique_id", how="left")

    return out.drop("_row_num","_batch_id")

In [0]:
# Create a temp dataframe with only the fields needed for the one-liner calculation
temp_df = usecases_goinglive_next.select(
    "unique_id",
    "usecase_name",
    "usecase_description",
    "demand_plan_stage_next_steps",
    "implementation_notes",
)

# Generate a one-liner summary for each use case using AI summarization
temp_df = batch_apply_expr(
    temp_df,
    "one_liner",
    "ai_summarize(concat_ws(' ', usecase_name, usecase_description, demand_plan_stage_next_steps, implementation_notes))",
    batch_size=100
)

# Join the one_liner result back to the main dataframe using unique_id
usecases_goinglive_next = usecases_goinglive_next.join(
    temp_df.select("unique_id", "one_liner"),
    on="unique_id",
    how="left"
)

usecases_goinglive_next.count()  # Force materialization

# Display the field manager, account name, use case name, and the generated one-liner summary
display(
    usecases_goinglive_next
        .orderBy(col("estimated_monthly_dollar_dbus").desc())
         .select(
            "field_manager",
            "account_name",
            "usecase_name",
            "estimated_monthly_dollar_dbus",
            "one_liner"
        ).limit(15)
)

In [0]:
# Define the confidence prompt outside the expression for clarity
ai_prompt_confidence = escape_prompt(
    "Using the MEDDPICC framework, and the following context, output a confidence score (0-100) for successful go-live. Only output the number that is multiple of 10.  Context: "
)

# Create a temp dataframe with only the fields needed to calculate confidence
temp_conf_df = usecases_goinglive_next.select(
    "unique_id",
    "usecase_description",
    "demand_plan_stage_next_steps",
    "implementation_notes"
)

# Use batch_apply_expr to batch the ai_query calls for confidence calculation
temp_conf_df = batch_apply_expr(
    temp_conf_df,
    "confidence",
    f"ai_query('databricks-gpt-oss-120b', concat('{ai_prompt_confidence}', coalesce(trim(usecase_description),''), ' ', coalesce(trim(demand_plan_stage_next_steps),''), ' ', coalesce(trim(implementation_notes),'')))",
    batch_size=100
)

# Join the calculated confidence back to usecases_goinglive_next
usecases_goinglive_next = usecases_goinglive_next.join(
    temp_conf_df.select("unique_id", "confidence"),
    on="unique_id",
    how="left"
)
usecases_goinglive_next.count()  # Force materialization

display(
    usecases_goinglive_next
        .orderBy(col("estimated_monthly_dollar_dbus").desc())
        .select(
            "field_manager",
            "account_name",
           "usecase_name",
            "estimated_monthly_dollar_dbus",
            "one_liner",
            "confidence"
        ).limit(15)
)

In [0]:
# Generate an AI prompt for extracting the top 3 confidence drivers using the MEDDPICC framework
ai_prompt_top3drivers = escape_prompt(
    "Using the MEDDPICC framework, and given the use case context, output a string that lists the top 3 drivers (factors raising or lowering confidence). Present them in the format: 1. <driver1> ||| 2. <driver2> ||| 3. <driver3> |||. Each driver should include a short, executive-level explanation. Use plain text. Context: "
)

# Create a temp dataframe with only the fields needed for the top3drivers calculation
temp_top3drivers_df = usecases_goinglive_next.select(
    "unique_id",
    "usecase_description",
    "demand_plan_stage_next_steps",
    "implementation_notes"
)

# Apply the AI model in batches to generate the top 3 drivers for each use case
temp_top3drivers_df = batch_apply_expr(
    temp_top3drivers_df,
    "top3drivers",
    f"""ai_query('databricks-gpt-oss-120b', concat('{ai_prompt_top3drivers}', concat_ws(' ', usecase_description, demand_plan_stage_next_steps, implementation_notes)))""",
    batch_size=100
)
# Display the results with the top 3 drivers included
display(temp_top3drivers_df.limit(15))

In [0]:
# Extract the top 3 confidence drivers from the 'top3drivers' string using regex and trim whitespace
from pyspark.sql.functions import col, trim, regexp_extract, when

temp_top3drivers_df = temp_top3drivers_df.withColumn(
    "driver_1", trim(regexp_extract(col("top3drivers"), r"1\.\s*(.*?)\s*\|\|\|", 1))
).withColumn(
    "driver_2", trim(regexp_extract(col("top3drivers"), r"2\.\s*(.*?)\s*\|\|\|", 1))
).withColumn(
    "driver_3", trim(regexp_extract(col("top3drivers"), r"3\.\s*(.*?)\s*\|\|\|", 1))
)

# Display the field manager, use case name, confidence score, and the extracted top 3 drivers
display(temp_top3drivers_df.limit(15))

In [0]:
usecases_goinglive_next = usecases_goinglive_next.join(
    temp_top3drivers_df.select("unique_id", "driver_1", "driver_2", "driver_3"),
    on="unique_id",
    how="left"
)

usecases_goinglive_next.count()  # Force materialization

display(
   usecases_goinglive_next
       .orderBy(col("estimated_monthly_dollar_dbus").desc())
       .select(
            "field_manager",
            "account_name",
            "usecase_name",
            "estimated_monthly_dollar_dbus",
            "confidence",
            "driver_1",
            "driver_2",
            "driver_3"
        ).limit(15)
)

In [0]:
from pyspark.sql.functions import when, lit, trim, regexp_extract, col, expr

# Define the AI prompt for recommending an action based on MEDDPICC and confidence score
ai_prompt_recommend_action = escape_prompt(
"""Using MEDDPICC and the given confidence score (0–100), return exactly one of: 
Proceed: <brief reason>
Pivot: <brief reason>
Walk: <brief reason>
Only output one line in this format.
Dont mention the confidence score or MEDDPICC in the output, just the reason.
If you output anything else, regenerate until compliant. 
Context: """
)

# Create a temp dataframe with only the fields needed for the recommend_action calculation
temp_recommend_action_df = usecases_goinglive_next.select(
    "unique_id",
    "confidence",
    "usecase_description",
    "demand_plan_stage_next_steps",
    "implementation_notes"
)

# Apply the AI model in batches to generate a recommended action and explanation for each use case
temp_recommend_action_df = batch_apply_expr(
    temp_recommend_action_df,
    "recommend_action_full",
    f"""ai_query('databricks-gpt-oss-120b', concat('{ai_prompt_recommend_action}', ' Confidence: ', cast(confidence as string), '. ', concat_ws(' ', usecase_description, demand_plan_stage_next_steps, implementation_notes)))""",
    batch_size=100
)

display(temp_recommend_action_df.limit(15))

In [0]:
# Extract the recommended action (Proceed, Pivot, Walk) from the AI output
temp_recommend_action_df = temp_recommend_action_df.withColumn(
    "recommend_action",
    regexp_extract(col("recommend_action_full"), r"^(?i)\s*(Proceed|Pivot|Walk)", 1)
)
# If the action is missing or empty, set it to "cant tell"
temp_recommend_action_df = temp_recommend_action_df.withColumn(
    "recommend_action",
    when((col("recommend_action").isNull()) | (trim(col("recommend_action")) == ""), lit("cant tell")).otherwise(col("recommend_action"))
)
# Extract the explanation for the recommended action
temp_recommend_action_df = temp_recommend_action_df.withColumn(
    "recommend_action_explanation",
    trim(regexp_extract(col("recommend_action_full"), r"^(?i)(?:Proceed|Pivot|Walk)\s*:\s*(.*)$", 1))
)

# Join recommend_action and recommend_action_explanation to usecases_goinglive_next by unique_id
usecases_goinglive_next = usecases_goinglive_next.join(
    temp_recommend_action_df.select("unique_id", "recommend_action", "recommend_action_explanation"),
    on="unique_id",
    how="left"
)

usecases_goinglive_next.count()  # Force materialization

# Display the relevant columns including the recommended action and its explanation
#display(
#    usecases_goinglive_next
#    .orderBy(col("estimated_monthly_dollar_dbus").desc())
#    .select(
#        "field_manager",
#        "account_name",
#        "usecase_name",
#        "estimated_monthly_dollar_dbus",
#        "recommend_action",
#        "recommend_action_explanation"
#    )
#)

In [0]:
from pyspark.sql.functions import col

# Columns to persist for the usecases_goinglive_next table
cols_to_persist = [
    "snapshot_date", 
    "sales_subregion_level_1",
    "sales_subregion_level_2",
    "sales_subregion_level_3",
    "field_manager",
    "account_name", 
    "usecase_name", 
    "account_executive",
    "account_solution_architect",
    "stage",
    "target_live_date",
    "onboarding_date",
    "estimated_monthly_dollar_dbus", 
    "one_liner",
    "confidence",
    "recommend_action",
    "recommend_action_explanation",
    "driver_1",
    "driver_2",
    "driver_3",
]

spark.conf.set("spark.databricks.execution.timeout", "14400")

# Drop the existing table if it exists before overwriting
spark.sql("DROP TABLE IF EXISTS users.luis_herrera.usecases_goinglive_next")

# Persist the selected columns to the target table
usecases_goinglive_next.select(cols_to_persist) \
    .write.mode("overwrite") \
    .saveAsTable("users.luis_herrera.usecases_goinglive_next")

In [0]:
spark.sql("OPTIMIZE users.luis_herrera.usecases_goinglive_next")

## # UC likely to slip 

In [0]:
from pyspark.sql.functions import col, trim, expr, current_date, add_months

# Select use cases likely to slip: confidence < 80, target_live_date in next 6 months, and recommend_action is Pivot or Proceed
slip_candidates_df = usecases_goinglive_next.filter(
    (col("confidence") < 80) &
    (col("target_live_date").between(current_date(), add_months(current_date(), 6))) &
    (trim(col("recommend_action")).isin(["Pivot", "Proceed"]))
)

#display(
#    slip_candidates_df
#    .orderBy(col("estimated_monthly_dollar_dbus").desc())
#    .select(
#        "field_manager",
#        "usecase_name",
#        "confidence",
#        "target_live_date",
#        "estimated_monthly_dollar_dbus",
#        "recommend_action_explanation"
#    ).limit(15)
#)

In [0]:
# Define slip categories and their descriptions for AI classification
slip_categories = [
    ("Technical", "Use cases may slip due to technical limitations, blockers, or unresolved challenges such as architectural complexity, lack of required features, or unresolved bugs that impede progress."),
    ("Business", "Business priorities, shifting organizational focus, lack of executive sponsorship, or unclear value proposition can deprioritize or stall use cases."),
    ("Stakeholder", "Insufficient stakeholder engagement, lack of buy-in, misalignment between teams, or changes in key personnel can delay or derail use case progress."),
    ("Budget", "Budget constraints, delayed funding approvals, or unexpected cost overruns can prevent necessary resources from being allocated, causing slippage."),
    ("Project Timelines", "Aggressive or unrealistic project timelines, missed deadlines, or dependencies on other projects can result in schedule slippage."),
    ("Data", "Delays in data access, poor data quality, data privacy concerns, or complex data integration requirements can block or slow down use case delivery."),
    ("Integration", "Challenges integrating with existing systems, APIs, or tools, or dependencies on third-party integrations, can introduce delays."),
    ("External dependencies", "Reliance on third parties, vendors, or external teams for critical deliverables or approvals can introduce uncertainty and delays."),
    ("Partner/Hyperscaler", "Issues related to technology partners or hyperscalers, such as lack of region support, limited partner capabilities, or changes in partner roadmaps, can impact timelines."),
    ("Competition", "Competitive pressures, market changes, or actions by competitors may shift priorities or require re-evaluation of the use case, causing delays."),
    ("Other", "Any other reasons not covered above, including unforeseen events, regulatory changes, or unique customer-specific challenges.")
]
array_str = ', '.join([f'"{c[0]}"' for c in slip_categories])


In [0]:
# Persist slip_categories as a table for later use
spark.conf.set("spark.databricks.execution.timeout", "14400")
spark.sql("DROP TABLE IF EXISTS users.luis_herrera.slipage_categories")
slip_categories_df = spark.createDataFrame(slip_categories, ["category", "description"])
slip_categories_df.write.mode("overwrite").saveAsTable("users.luis_herrera.slipage_categories")

In [0]:
from pyspark.sql.functions import when, col, lit, expr


# Create a temp dataframe with only the fields needed for the category calculation
temp_slip_category_df = slip_candidates_df.select(
    "unique_id",
    "recommend_action_explanation"
)

# Use batch_apply_expr to calculate the category in batches
temp_slip_category_batched_df = batch_apply_expr(
    temp_slip_category_df,
    "category",
    f"ai_classify(recommend_action_explanation, array({array_str}))",
    batch_size=100
)

temp_slip_category_batched_df = temp_slip_category_batched_df.withColumn(
    "category",
    when(col("category").isNull() | (trim(col("category")) == ""), lit("Other")).otherwise(col("category"))
)

# Join the category result back to slip_candidates_df
slip_candidates_df = slip_candidates_df.join(
    temp_slip_category_batched_df.select("unique_id", "category"),
    on="unique_id",
    how="left"
)

#display(
#    slip_candidates_final_df
#    .orderBy(col("estimated_monthly_dollar_dbus").desc())
#   .select(
#        "field_manager",
#        "usecase_name",
#        "confidence",
#        "target_live_date",
#        "estimated_monthly_dollar_dbus",
#        "category"
#    ).limit(15)
#)

In [0]:
from pyspark.sql.functions import expr, trim, coalesce, when, col, lit

# AI prompt for slippage recommendation
ai_prompt_slippage = (
    "Given the following use case description, next steps, and implementation notes, recommend one specific action to mitigate or eliminate slippage. "
    "Base your recommendation on exactly one of the following categories: "
    "Technical, Business, Stakeholder, Budget, Project Timelines, Data, Integration, External dependencies, Partner/Hyperscaler, Competition, Other. "
    "If no clear mitigation or elimination action is possible, output exactly: No clear way of eliminating or mitigating slippage. "
    "Only output the recommendation as a concise sentence."
)

# Create a temp dataframe with only the fields needed for the slippage recommendation
temp_slip_df = slip_candidates_df.select(
    "unique_id",
    "usecase_description",
    "demand_plan_stage_next_steps",
    "implementation_notes"
)

# Use batch_apply_expr to batch the ai_query calls for slippage recommendation
temp_slip_batched_df = batch_apply_expr(
    temp_slip_df,
    "slippage_recommendation",
    f"""ai_query('databricks-gpt-oss-120b', \
        concat('{ai_prompt_slippage}', \
               ' Description: ', coalesce(trim(usecase_description),''), \
               ' Next Steps: ',  coalesce(trim(demand_plan_stage_next_steps), ''), \
               ' Implementation Notes: ', coalesce(trim(implementation_notes), '')\
        )\
    )""",
    batch_size=100
)

temp_slip_batched_df = temp_slip_batched_df.withColumn(
    "slippage_recommendation",
    when(col("slippage_recommendation").isNull() | (trim(col("slippage_recommendation")) == ""), lit("No clear way of eliminating or mitigating slippage")).otherwise(col("slippage_recommendation"))
)

# Join the slippage recommendation back to slip_candidates_df using unique_id
slip_candidates_df = slip_candidates_df.join(
    temp_slip_batched_df.select("unique_id", "slippage_recommendation"),
    on="unique_id",
    how="left"
)

#display(
#    slip_candidates_df.select(
#        "field_manager",
#        "usecase_name",
#        "confidence",
#        "target_live_date",
#        "category",
#        "slippage_recommendation"
#    ).limit(15)
#)

In [0]:
cols_for_slippage = [
    "snapshot_date", 
    "sales_subregion_level_1",
    "sales_subregion_level_2",
    "sales_subregion_level_3",
    "field_manager",
    "account_name", 
    "usecase_name", 
    "account_executive",
    "account_solution_architect",
    "stage",
    "target_live_date",
    "onboarding_date",
    "estimated_monthly_dollar_dbus", 
    "one_liner",
    "category",
    "slippage_recommendation"
]

slip_candidates_final_df = slip_candidates_df.select(cols_for_slippage)
#display(slip_candidates_final_df.limit(15))

In [0]:
spark.conf.set("spark.databricks.execution.timeout", "14400")
spark.sql("DROP TABLE IF EXISTS users.luis_herrera.slip_candidates")
slip_candidates_final_df.write.mode("overwrite").option("mergeSchema", "false").saveAsTable("users.luis_herrera.slip_candidates")

## UC that can be potentially accelerated

In [0]:
from pyspark.sql.functions import col, trim, expr, current_date, add_months

# Filter use cases with target live date in next 6 months and action is Pivot or Proceed
accel_candidates_df = usecases_goinglive_next.filter(
    (col("target_live_date").between(current_date(), add_months(current_date(), 6))) &
    (trim(col("recommend_action")).isin(["Pivot", "Proceed"]))
)
# Display relevant columns for acceleration candidates
#display(
#    accel_candidates_df.select(
#        "field_manager",
#        "usecase_name",
#        "confidence",
#        "target_live_date",
#        "recommend_action"
#    ).limit(15)
#)

In [0]:
from pyspark.sql.functions import expr, trim, coalesce

# AI prompt for acceleration recommendation
ai_prompt_accelerate = (
    "Given the following use case description, next steps and implementation notes, recommend a specific action to accelerate progress. "
    "Base your recommendation on one of these categories: Funding, Implementation Planning, Resource Allocation, Stakeholder Engagement, "
    "Technical Optimization, Risk Mitigation, Partnership Leverage, Process Streamlining, Monitoring and Support, Innovation and Tools. "
    "If you cannot identify a clear acceleration opportunity, output exactly: No clear acceleration opportunity. "
    "Only output the recommendation as a concise sentence."
)

# Create a temp dataframe with only the fields needed for the acceleration recommendation
temp_accel_df = accel_candidates_df.select(
    "unique_id",
    "usecase_description",
    "demand_plan_stage_next_steps",
    "implementation_notes"
)

# Use batch_apply_expr to batch the ai_query calls for acceleration recommendation
temp_accel_df = batch_apply_expr(
    temp_accel_df,
    "acceleration_recommendation",
    f"""ai_query('databricks-gpt-oss-120b', \
        concat('{ai_prompt_accelerate}', \
               ' Description: ', coalesce(trim(usecase_description),''), \
               ' Next Steps: ',  coalesce(trim(demand_plan_stage_next_steps), ''), \
               ' Implementation Notes: ', coalesce(trim(implementation_notes), '')\
        )\
    )""",
    batch_size=100
)



In [0]:
# Join the acceleration recommendation back to accel_candidates_df
accel_candidates_df = accel_candidates_df.join(
    temp_accel_df.select("unique_id", "acceleration_recommendation"),
    on="unique_id",
    how="left"
)

#display(
#    accel_candidates_df.select(
#        "field_manager",
#        "usecase_name",
#        "confidence",
#        "target_live_date",
#        "acceleration_recommendation"
#    ).limit(15)
#)

In [0]:
# Define categories and their descriptions for AI classification
accel_categories = [
    ("Funding", "Accelerate the use case by securing funding, investment, or financial approvals to support project progression."),
    ("Implementation Planning", "Accelerate the use case by finalizing project plans, setting clear milestones, and establishing a delivery timeline to ensure timely completion."),
    ("Resource Allocation", "Accelerate the use case by allocating sufficient resources, including personnel, infrastructure, and tools, to support project requirements."),
    ("Stakeholder Engagement", "Accelerate the use case by engaging with key stakeholders, including customers, partners, and executives, to build momentum, resolve blockers, and ensure alignment."),
    ("Technical Optimization", "Accelerate the use case by optimizing technical aspects, such as data migration, integration, and performance, to improve overall efficiency and speed."),
    ("Risk Mitigation", "Accelerate the use case by identifying and mitigating potential risks, including technical debt, skill gaps, and interoperability issues, to ensure project stability and progress."),
    ("Partnership Leverage", "Accelerate the use case by leveraging partnerships, collaborations, and joint initiatives to access expertise, resources, and funding opportunities."),
    ("Process Streamlining", "Accelerate the use case by streamlining processes, including customer enablement, sales kits, and account activation, to reduce delays and improve overall velocity."),
    ("Monitoring and Support", "Accelerate the use case by providing regular monitoring, support, and feedback to ensure project progress, identify roadblocks, and address issues promptly."),
    ("Innovation and Tools", "Accelerate the use case by leveraging innovative solutions, tools, and technologies, such as AI, Lakebridge, and Tredence's migration accelerator, to expedite project delivery and improve outcomes.")
]
array_str = ', '.join([f'"{c[0]}"' for c in accel_categories])

In [0]:
spark.sql("DROP TABLE IF EXISTS users.luis_herrera.acceleration_categories")
accel_categories_df = spark.createDataFrame(accel_categories, ["category", "description"])
accel_categories_df.write.mode("overwrite").saveAsTable("users.luis_herrera.acceleration_categories")

In [0]:
# Classify the AI-generated recommendation into one of the defined categories
acceleration_final_df =  accel_candidates_df.withColumn(
        "category",
        expr(f"ai_classify(acceleration_recommendation, array({array_str}))")
      )
# display(acceleration_final_df
#     .orderBy(col("estimated_monthly_dollar_dbus").desc())
#     .select(
#         "field_manager",
#         "usecase_name",
#         "confidence",
#         "target_live_date",
#         "estimated_monthly_dollar_dbus",
#         "category",
#         "acceleration_recommendation"
#     ).limit(15)
# )

In [0]:
cols_for_accel = [
    "snapshot_date", 
    "sales_subregion_level_1",
    "sales_subregion_level_2",
    "sales_subregion_level_3",
    "field_manager",
    "account_name", 
    "usecase_name", 
    "account_executive",
    "account_solution_architect",
    "stage",
    "target_live_date",
    "onboarding_date",
    "estimated_monthly_dollar_dbus", 
    "one_liner",
    "category",
    "acceleration_recommendation"   
]


acceleration_final_df = acceleration_final_df.select(cols_for_accel)

In [0]:
spark.conf.set("spark.databricks.execution.timeout", "14400")
spark.sql("DROP TABLE IF EXISTS users.luis_herrera.acceleration_candidates")
acceleration_final_df.write.mode("overwrite").option("mergeSchema", "false").saveAsTable("users.luis_herrera.acceleration_candidates")

In [0]:
# Document the table and fields using table and column comments to later use within Genie

# Table comment for usecases_goinglive_next
spark.sql("""
ALTER TABLE users.luis_herrera.usecases_goinglive_next
SET TBLPROPERTIES (
  'comment' = 'Table containing use cases expected to go live within next 6 monthns, with MEDDPICC-based AI recommendations and related metadata.'
)
""")

# Column comments for usecases_goinglive_next
column_comments = {
    "snapshot_date": "Date of salesforce data snapshot.",
    "sales_subregion_level_1": "Top-level sales subregion. Possible values: Central, BNLX & Nordics, UKI, SEMEA, Emerging, Greenfield, DNB, FINS, HLS.",
    "sales_subregion_level_2": "Second-level sales subregion. Possible values: Germany Enterprise, Germany Strategic Non-Regulated, Switzerland, Benelux_Non Regulated, Germany Strategic Regulated, Nordics, UK TTL, MEI & CME, UK HLS, PBS & RCG, Spain, IMEA, France Product Industries, UK FSI & Public Sector, Benelux_Regulated, UK Geo Strat & Named Core, UK&I, France Service Industries, Nordics & Benelux, Central, Italy, Emerging_Greenfield, DNB Hunter, Southern, Nordics_Region, Enterprise_Greenfield_Northern, Emerging Greenfield, France Hunter, Greenfield, FINS ProServ & FinTech SLM, HLS LS.",
    "sales_subregion_level_3": "Third-level sales subregion.",
    "field_manager": "Field manager responsible for the account.",
    "account_name": "Customer account name.",
    "usecase_name": "Name of the use case.",
    "account_executive": "Account executive assigned.",
    "account_solution_architect": "Solution architect assigned.",
    "stage": "Current sales or implementation stage.",
    "target_live_date" : "Planned go-live date for the use case.",
    "onboarding_date" : "Planned onboarding date for the use case",
    "estimated_monthly_dollar_dbus": "Estimated monthly DBUs in dollars.",
    "one_liner": "Brief summary of the use case.",
    "confidence": "AI-assigned confidence score (0-100) for go-live.",
    "recommend_action": "AI-recommended action: Proceed, Pivot, Walk, or cant tell.",
    "recommend_action_explanation": "Explanation for the recommended action.",
    "driver_1": "Primary business or technical driver.",
    "driver_2": "Secondary business or technical driver.",
    "driver_3": "Tertiary business or technical driver."
}

for col_name, comment in column_comments.items():
    spark.sql(f"""
    ALTER TABLE users.luis_herrera.usecases_goinglive_next
    ALTER COLUMN {col_name} COMMENT '{comment}'
    """)

# Table comment for slip_candidates
spark.sql("""
ALTER TABLE users.luis_herrera.slip_candidates
SET TBLPROPERTIES (
  'comment' = 'Use cases at risk of slippage within next 6 monthn, with AI-classified slippage category and mitigation recommendations.'
)
""")

# Column comments for slip_candidates
slip_column_comments = {
    "snapshot_date": "Date of data snapshot.",
    "sales_subregion_level_1": "Top-level sales subregion. Possible values: Central, BNLX & Nordics, UKI, SEMEA, Emerging, Greenfield, DNB, FINS, HLS.",
    "sales_subregion_level_2": "Second-level sales subregion. Possible values: Germany Enterprise, Germany Strategic Non-Regulated, Switzerland, Benelux_Non Regulated, Germany Strategic Regulated, Nordics, UK TTL, MEI & CME, UK HLS, PBS & RCG, Spain, IMEA, France Product Industries, UK FSI & Public Sector, Benelux_Regulated, UK Geo Strat & Named Core, UK&I, France Service Industries, Nordics & Benelux, Central, Italy, Emerging_Greenfield, DNB Hunter, Southern, Nordics_Region, Enterprise_Greenfield_Northern, Emerging Greenfield, France Hunter, Greenfield, FINS ProServ & FinTech SLM, HLS LS.",
    "sales_subregion_level_3": "Third-level sales subregion.",
    "field_manager": "Field manager responsible for the account.",
    "account_name": "Customer account name.",
    "usecase_name": "Name of the use case.",
    "account_executive": "Account executive assigned.",
    "account_solution_architect": "Solution architect assigned.",
    "stage": "Current sales or implementation stage.",
    "target_live_date": "Planned go-live date for the use case.",
    "estimated_monthly_dollar_dbus": "Estimated monthly DBUs in dollars.",
    "one_liner": "Brief summary of the use case.",
    "category": "AI-classified slippage category.",
    "slippage_recommendation": "AI-generated recommendation to mitigate or eliminate slippage."
}

for col_name, comment in slip_column_comments.items():
    spark.sql(f"""
    ALTER TABLE users.luis_herrera.slip_candidates
    ALTER COLUMN {col_name} COMMENT '{comment}'
    """)

# Table comment for acceleration_candidates
spark.sql("""
ALTER TABLE users.luis_herrera.acceleration_candidates
SET TBLPROPERTIES (
  'comment' = 'Use cases with acceleration opportunities within next 6 monthn, including AI-generated recommendations and classified categories.'
)
""")

# Column comments for acceleration_candidates
accel_column_comments = {
    "snapshot_date": "Date of data snapshot.",
    "sales_subregion_level_1": "Top-level sales subregion. Possible values: Central, BNLX & Nordics, UKI, SEMEA, Emerging, Greenfield, DNB, FINS, HLS.",
    "sales_subregion_level_2": "Second-level sales subregion. Possible values: Germany Enterprise, Germany Strategic Non-Regulated, Switzerland, Benelux_Non Regulated, Germany Strategic Regulated, Nordics, UK TTL, MEI & CME, UK HLS, PBS & RCG, Spain, IMEA, France Product Industries, UK FSI & Public Sector, Benelux_Regulated, UK Geo Strat & Named Core, UK&I, France Service Industries, Nordics & Benelux, Central, Italy, Emerging_Greenfield, DNB Hunter, Southern, Nordics_Region, Enterprise_Greenfield_Northern, Emerging Greenfield, France Hunter, Greenfield, FINS ProServ & FinTech SLM, HLS LS.",
    "sales_subregion_level_3": "Third-level sales subregion.",
    "field_manager": "Field manager responsible for the account.",
    "account_name": "Customer account name.",
    "usecase_name": "Name of the use case.",
    "account_executive": "Account executive assigned.",
    "account_solution_architect": "Solution architect assigned.",
    "stage": "Current sales or implementation stage.",
    "target_live_date": "Planned go-live date for the use case.",
    "estimated_monthly_dollar_dbus": "Estimated monthly DBUs in dollars.",
    "one_liner": "Brief summary of the use case.",
    "category": "AI-classified acceleration category.",
    "acceleration_recommendation": "AI-generated recommendation to accelerate progress."
}

for col_name, comment in accel_column_comments.items():
    spark.sql(f"""
    ALTER TABLE users.luis_herrera.acceleration_candidates
    ALTER COLUMN {col_name} COMMENT '{comment}'
    """)

# Table comment for slip_categories
spark.sql("""
ALTER TABLE users.luis_herrera.slipage_categories
SET TBLPROPERTIES (
  'comment' = 'Reference table of slippage categories and their descriptions for AI classification of use case slippage risk.'
)
""")

# Column comments for slip_categories
slip_cat_column_comments = {
    "category": "Name of the slippage category.",
    "description": "Description of the slippage category."
}

for col_name, comment in slip_cat_column_comments.items():
    spark.sql(f"""
    ALTER TABLE users.luis_herrera.slipage_categories
    ALTER COLUMN {col_name} COMMENT '{comment}'
    """)

# Table comment for acceleration_categories
spark.sql("""
ALTER TABLE users.luis_herrera.acceleration_categories
SET TBLPROPERTIES (
  'comment' = 'Reference table of acceleration categories and their descriptions for AI classification of use case acceleration opportunities.'
)
""")

# Column comments for acceleration_categories
accel_cat_column_comments = {
    "category": "Name of the acceleration category.",
    "description": "Description of the acceleration category."
}

for col_name, comment in accel_cat_column_comments.items():
    spark.sql(f"""
    ALTER TABLE users.luis_herrera.acceleration_categories
    ALTER COLUMN {col_name} COMMENT '{comment}'
    """)